In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def prd_calc(prd,Data):
    
    # Takes prd (integer) is the period were we need to find the data   #
    # Takes Data (Dataframe) is the data                                #
    # Return the average of every sector in this period based on spread #                                                         #
    dist=(Data.ind).value_counts()
    dist.index
    
    DataDay_one=[]
    for i in range(0,len(dist.index)):
        Day=Data.loc[Data['prd']==prd]
        Sector=Day.loc[Day['ind']==dist.index[i]]
        tret=Sector['tret']
        tret.to_numpy()
        avg_tret=np.average(tret)
        DataDay_one.append(avg_tret)
        
    return DataDay_one

In [3]:
def Values_from_correlation(ind1,ind2,corr):
    return corr[ind1][ind2]

In [4]:
def calculate_matricies(tick_ind,corr):
    corr_by_int=[]
    for int_i,sector_i in zip(tick_ind['int_tick'],tick_ind['ind']):
        lines=[]
        for int_j , sector_j in zip(tick_ind['int_tick'],tick_ind['ind']):
            lines.append(Values_from_correlation(sector_i,sector_j,corr))
        corr_by_int.append(lines)
    corr_by_int=pd.DataFrame(corr_by_int)
    corr_by_int.index=list(tick_ind['int_tick'])
    corr_by_int.columns=list(tick_ind['int_tick'])
    return corr_by_int

In [5]:
# import the data
Data=pd.read_csv('preproccesed.csv')
Data=Data.drop('Unnamed: 0',axis=1)

# Keep the data valuable for correlation
prd=Data.prd
int_tick=Data.int_tick
ind=Data.ind
tret=Data.tret
formal=pd.concat([int_tick,prd,ind,tret],axis=1)

In [8]:
# list of periods
periods=list((formal.loc[formal['int_tick']==int_tick[0]]).prd)

# for every period
SectPrd=[]
for i in range(0,len(periods)):
    SectPrd.append(prd_calc(periods[i],formal))

Sector_series=np.array(SectPrd) 
Sector_series=pd.DataFrame(Sector_series)

# Names of industries
Names_of_inds=list(((formal.ind).value_counts()).index)
Sector_series.columns=Names_of_inds


In [9]:
Sector_series

,Utility,FBT,Telco,EandP,GasDis,Media,Mtls,PapPrint,Health,CapG,...,Chem,Leisure,Elec,Svces,Products,Tech,FoodRtl,Aero,Auto,Software
0,-0.000639,0.000792,0.001275,-0.000555,-0.000511,0.001675,-0.001265,0.000107,0.000350,0.000158,...,0.000490,0.002722,0.000607,0.002892,-0.000003,0.000283,-0.000367,0.000741,0.000247,0.000629
1,-0.000198,0.000531,0.001126,0.000312,0.000376,-0.000548,-0.000202,0.001269,0.000906,0.000180,...,0.001812,0.003086,0.000592,0.001979,0.000908,0.000982,0.000387,0.001318,0.001276,0.000559
2,-0.000025,0.000407,0.000816,0.000324,0.000323,-0.000145,-0.000011,0.000936,0.000614,0.000181,...,0.001160,0.001936,0.000479,0.001404,0.000627,0.000652,0.000325,0.000878,0.000891,0.000400
3,0.000044,0.000406,0.000874,0.001880,0.000626,-0.000022,0.001160,0.001199,0.000660,0.000147,...,0.000778,0.001389,0.001088,0.001139,0.000562,0.000671,0.000150,0.000661,0.001264,0.000255
4,0.000206,0.000113,0.000454,0.001561,0.000401,0.000316,0.001671,0.001301,0.000710,0.000192,...,0.000062,0.001603,0.002675,0.001280,0.000481,0.000615,0.000950,0.000729,0.001516,0.000134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0.000683,0.000508,0.001122,0.001623,0.000903,0.001447,0.001993,0.002515,0.001123,0.000579,...,0.000312,0.001810,0.001042,0.001756,0.000236,0.000506,0.001497,0.001209,0.001109,0.000305
77,0.000617,0.000492,0.001094,0.001618,0.000832,0.001336,0.002077,0.002600,0.001293,0.000593,...,0.000263,0.002162,0.000926,0.001682,0.000195,0.000662,0.001433,0.001212,0.001009,0.000268
78,0.000580,0.000479,0.001082,0.001633,0.000817,0.001270,0.002219,0.002560,0.001249,0.000536,...,0.000522,0.002874,0.000791,0.001610,0.000167,0.000779,0.001981,0.001247,0.000958,0.000226
79,0.000707,0.000560,0.001081,0.001628,0.000966,0.001216,0.002188,0.002338,0.001205,0.000451,...,0.000322,0.002990,0.000747,0.000989,0.000139,0.000774,0.001863,0.001255,0.000885,0.000349


In [10]:
# calculate correlation by window
corr_prd=[]
steps=[]
end=0
for start in range(0,len(Sector_series)):
    if end<len(Sector_series)-1:
        # for every window
        end=start+10
        step=Sector_series.loc[start:end]
        print(end)
        steps.append(step)
        step_corr=step.corr()
        # Make zeros those below zero
        step_corr[step_corr<0]=0
        # Normalize correlation map
        step_corr = np.array(step_corr)   
        step_corr=(step_corr-step_corr.min())/(step_corr.max()-step_corr.min()) 
        # Take the 1-corr
        step_corr=pd.DataFrame(step_corr)
        step_corr=1-step_corr

        step_corr.columns=Names_of_inds
        step_corr.index=Names_of_inds

        corr_prd.append(step_corr)

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [12]:
sample_day=formal.loc[formal['prd']==periods[0]]
int_tick=sample_day.int_tick
ind=sample_day.ind
int_tick_ind=pd.concat([int_tick,ind],axis=1)
int_tick_ind=int_tick_ind.reset_index(drop=True)

In [13]:
time_series_corr=[]
iter_=1
for corr in corr_prd:
    print(iter_)
    time_series_corr.append(calculate_matricies(int_tick_ind,corr))
    iter_=iter_+1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71


In [14]:
use_periods=periods[10:end+1]

In [15]:
for prd,corr in zip(use_periods,time_series_corr):
    corr.to_csv(str(prd)+' int-tret.csv')